In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

dataset = pd.read_csv('datasets/bests/PCA_10_CNN_19_256_MAX.csv', encoding='utf-8')

X = MinMaxScaler().fit_transform(dataset[dataset.columns[1:]])
y = dataset.classe

In [2]:
## Carregando o StackingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier

# Algoritmo para fazer a fusão
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.metrics import confusion_matrix

# Separando dataset em duas partes: treinamento e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [3]:
bests_five = [
    ("DT", DecisionTreeClassifier(criterion="entropy", max_depth=9)),
    ("kNN", KNeighborsClassifier(n_neighbors=7, metric="euclidean")),
    (
        "MLP-Config-4",
        MLPClassifier(
            hidden_layer_sizes=516,
            activation="relu",
            solver="adam",
            max_iter=500,
            learning_rate_init=0.0001,
        ),
    ),
    (
        "MLP-Config-5",
        MLPClassifier(
            hidden_layer_sizes=258,
            activation="identity",
            solver="sgd",
            max_iter=1000,
            learning_rate_init=0.0001,
        ),
    ),
    ("NB", MultinomialNB(fit_prior=True, alpha=1.0)),
]

In [4]:
ten_classifiers = bests_five + [
    (
        "Bagging_MLP",
        BaggingClassifier(
            estimator=MLPClassifier(
                hidden_layer_sizes=516,
                activation="relu",
                solver="adam",
                max_iter=500,
                learning_rate_init=0.0001,
            ),
            n_estimators=20,
            max_features=0.5,
        ),
    ),
    (
        "AdaBoost_DT",
        AdaBoostClassifier(
            estimator=DecisionTreeClassifier(criterion="entropy", max_depth=9),
            n_estimators=30,
        ),
    ),
    (
        "RandomForest",
        RandomForestClassifier(n_estimators=100, criterion="gini"),
    ),
    (
        "MLP-Config-10",
        MLPClassifier(
            hidden_layer_sizes=258,
            activation="relu",
            solver="sgd",
            max_iter=500,
            learning_rate_init=0.1,
        )
    ),
    (
        "GaussianNB",
        GaussianNB(priors=None, var_smoothing=1e-09),
    ),
]

In [5]:
fifteen_classifiers = ten_classifiers + [
    (
        "MLP-Config-3",
        MLPClassifier(
            hidden_layer_sizes=(258, 128),
            activation="logistic",
            solver="sgd",
            max_iter=500,
            learning_rate_init=0.01,
        ),
    ),
    (
        "knn-9",
        KNeighborsClassifier(n_neighbors=9, metric="euclidean"),
    ),
    (
        "ComplementNB",
        ComplementNB(alpha=1.0, fit_prior=True, force_alpha=True),
    ),
    (
        "knn-1",
        KNeighborsClassifier(n_neighbors=1, metric="euclidean"),
    ),
    (
        "Bagging-kNN-7",
        BaggingClassifier(
            estimator=KNeighborsClassifier(n_neighbors=7, metric="euclidean"),
            n_estimators=10,
            max_features=0.5,
        ),
    )
]

In [6]:
twenty_classifiers = fifteen_classifiers + [
    (
        "RandomForest-Gini-10",
        RandomForestClassifier(n_estimators=10, criterion="gini"),
    ),
    (
        "AdaBoost-10-DT-6",
        AdaBoostClassifier(
            estimator=DecisionTreeClassifier(criterion="entropy", max_depth=6),
            n_estimators=10,
        ),
    ),
    (
        "knn-3",
        KNeighborsClassifier(n_neighbors=3, metric="euclidean"),
    ),
    (
        "Bagging-MLP-Config-10",
        BaggingClassifier(
            estimator=MLPClassifier(
                hidden_layer_sizes=258,
                activation="relu",
                solver="sgd",
                max_iter=500,
                learning_rate_init=0.1,
            ),
            n_estimators=10,
            max_features=0.5,
        ),
    ),
    (
        "Bagging-MultinomialNB",
        BaggingClassifier(
            estimator=MultinomialNB(fit_prior=True, alpha=1.0),
            n_estimators=10,
            max_features=0.5,
        ),
    ),
]

In [7]:
stk = StackingClassifier(estimators=bests_five, final_estimator=LogisticRegression())

stk.fit(X_train, y_train)
y_pred = stk.predict(X_test)

# Model Accuracy
acuracia = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acuracia)

/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/skl

Accuracy: 0.908


/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [8]:
stk = StackingClassifier(estimators=ten_classifiers, final_estimator=LogisticRegression())

stk.fit(X_train, y_train)
y_pred = stk.predict(X_test)

# Model Accuracy
acuracia = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acuracia)

/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/skl

Accuracy: 0.904


In [9]:
stk = StackingClassifier(estimators=fifteen_classifiers, final_estimator=LogisticRegression())

stk.fit(X_train, y_train)
y_pred = stk.predict(X_test)

# Model Accuracy
acuracia = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acuracia)

/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/skl

Accuracy: 0.912


In [10]:
stk = StackingClassifier(estimators=twenty_classifiers, final_estimator=LogisticRegression())

stk.fit(X_train, y_train)
y_pred = stk.predict(X_test)

# Model Accuracy
acuracia = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.3f" % acuracia)

/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/isaac/.local/lib/python3.10/site-packages/skl

Accuracy: 0.912
